In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

##########################################################################

J = 1.0

# initialize the configuration by randomly choosing the spin patterns at each site
def init_config_ising(L):
    return np.random.choice([1,-1], size = (L,L))

# calculate the energy difference after flipping the spin at a certain site
def delta_E(config, site):
    delta_E = 0.0
    L = np.shape(config)[0]
    for nbs in [(1,0),(-1,0),(0,1),(0,-1)]: # take summation over each neighborhood of this site, don't forget to multiply by 2
        delta_E += config[(site[0] + nbs[0])%L, (site[1] + nbs[1])%L] * J * config[site] * 2
    return delta_E

def MCMC_update(config, T):
    # implement of metropolis algorithm
    L = np.shape(config)[0]
    random_site = (np.random.randint(L), np.random.randint(L))
    energy = delta_E(config, random_site)
    r = np.random.rand() # random number sampled from U(0,1)
    alpha = np.exp(-energy/T) # alpha =  the proportion of probabilities
    if r < alpha or energy < 0:
        config[random_site] *= -1
    return config

def get_energy(config):
    L = np.shape(config)[0]
    energy = 0.0
    for i in range(L):
        for j in range(L):
            energy += - config[i,j] * config[(i+1)%L, j] - config[i,j] * config[i, (j+1)%L] # get the exact energy of a configuration
    return energy

def get_magn(config):
    magn = np.sum(config) # get the exact magnetization of a configuation
    return magn

##########################################################################

n_iter = 100000
L_list = [2, 4, 6]
T_list = np.linspace(1, 3.5, 100)
Tc = 2/np.log(1+np.sqrt(2))

def MCMC(L, T, n_iter, n_avg=10000): # implement of the whole MCMC iteration

    state = init_config_ising(L)

    for _ in range(n_iter):
        state = MCMC_update(state, T) # implement MCMC update for n_iter = 100000 times (following the suggestion given in the lecture note)

    energies = []
    magnetizations = []
    for _ in range(n_avg*10): # for calculation of the physical observations, further sample 10 * n_average configurations
        state = MCMC_update(state, T)
        sample_energy = get_energy(state)
        sample_magnetization = get_magn(state)
        energies.append(sample_energy)
        magnetizations.append(sample_magnetization)
        
    subsample_energies = np.random.choice(energies, size=n_avg)
    subsample_magnetizations = np.abs(np.random.choice(magnetizations, size=n_avg))
    avg_energy = np.mean(subsample_energies) / (L*L)
    avg_magnetization = np.mean(subsample_magnetizations) / (L*L)
    specific_heat = np.var(subsample_energies) / (T ** 2)
    susceptibility = np.var(subsample_magnetizations) / T

    return state, avg_energy, avg_magnetization, specific_heat, susceptibility # output the final physical observations of one MCMC iteration


# setting of the figures (5 subfigures)
fig = plt.figure(figsize=(15, 30))
ax_energy = fig.add_subplot(421)
ax_energy.set_xlabel(r'$T$')
ax_energy.set_ylabel(r'$\langle E\rangle$')
ax_magnetization = fig.add_subplot(422)
ax_magnetization.set_xlabel(r'$T$')
ax_magnetization.set_ylabel(r'$\langle M \rangle$')
ax_specific_heat = fig.add_subplot(423)
ax_specific_heat.set_xlabel(r'$T$')
ax_specific_heat.set_ylabel(r'$\langle C\rangle$')
ax_susceptibility = fig.add_subplot(424)
ax_susceptibility.set_xlabel(r'$T$')
ax_susceptibility.set_ylabel(r'$\langle\chi\rangle$')
ax_exponent = fig.add_subplot(212)
ax_exponent.set_xlabel(r'$N|T-T_c|$')
ax_exponent.set_ylabel(r'$N^{1/8}\langle M\rangle$')
ax_exponent.set_xscale('log') # logarithm coordinate in the critical exponent graph
ax_exponent.set_yscale('log') 

###### start MC iteration ######
for L in L_list:
    avg_energies=[]
    avg_magnetizations=[]
    specific_heats=[]
    susceptibilities=[]
    
    for T in tqdm(T_list):
        state, avg_energy, avg_magnetization, specific_heat, susceptibility = MCMC(L, T, n_iter)
        avg_energies.append(avg_energy)
        avg_magnetizations.append(avg_magnetization)
        specific_heats.append(specific_heat)
        susceptibilities.append(susceptibility)

    print('L = {}, iteration {} MC iteration done.'.format(L, n_iter))
    # plot
    ax_energy.scatter(T_list, avg_energies, label=f'L = {L}')
    ax_magnetization.scatter(T_list, avg_magnetizations, label=f'L = {L}')
    ax_specific_heat.scatter(T_list, np.array(specific_heats)/(L*L), label=f'L = {L}')
    ax_susceptibility.scatter(T_list, np.array(susceptibilities)/(L*L), label=f'L = {L}')
    ax_exponent.scatter(np.abs(T_list-Tc) * (L*L), np.array(avg_magnetizations) * ((L*L) ** (1/8)), label=f'L = {L}')
    
# exact solution (beta = 1/8, -7/8)
ax_exponent.plot(np.abs(T_list-Tc) * (L*L), (np.abs(T_list-Tc) * (L*L)) ** (1/8), label=r'exact at $T<T_c$')
ax_exponent.plot(np.linspace(0.1, 3.5-Tc) * (L*L), 10*(np.linspace(0.1, 3.5-Tc) * (L*L)) ** (-7/8), label=r'exact at $T>T_c$')
ax_energy.legend()
ax_magnetization.legend()
ax_specific_heat.legend()
ax_susceptibility.legend()
ax_exponent.legend()
plt.show()